In [7]:
import sys, os

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.append(project_root)
    
os.environ["TRANSFORMERS_NO_TF"] = "1"

import json 
import pandas as pd
from collections import OrderedDict
import re
from underthesea import sent_tokenize

from rag_model.model.RE.final_re import RE
from rag_model.model.NER.final_ner import NER
from shared_functions.global_functions import *

with open('D:/Study/Education/Projects/Group_Project/rag_model/model/RE/artifact/id2relation.json', 'r') as f:
    id2relation = json.load(f)

In [3]:
from vncorenlp import VnCoreNLP

ner_annotator = VnCoreNLP("D:/Study/Education/Projects/Group_Project/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g') 

ner = NER(
    model_path="D:/Study/Education/Projects/Group_Project/rag_model/model/NER/artifact/model_bilstm_crf.pt",
    token2idx_path="D:/Study/Education/Projects/Group_Project/rag_model/model/NER/artifact/token2idx.json",
    label2idx_path="D:/Study/Education/Projects/Group_Project/rag_model/model/NER/artifact/label2idx.json",
    annotator = ner_annotator
)

re_model = RE(checkpoint = 'D:/Study/Education/Projects/Group_Project/rag_model/model/RE/artifact/re_8_train_phobert_1_3.pth',
           use_phobert=True, id2relation=id2relation, encoder_layer=1, decoder_layer=3, use_rel_pos=False, freeze_train=False) #match the model configuration

In [5]:
check_mask = ['luật', 'thông', 'nghị', 'hiến', 'quyết', 'định', 'pháp', 'tư', 'điều', 'mục', 'phần', 'khoản']

structure_dict = {
    'chương': "chapter", ## and _(Roman_number)
    'điều': 'C', ## and _(decimal_number)
    'khoản': "P", ## and_(decimal_number)
    'điểm': 'SP', ## and _(decimal_number)
    'điểm': 'SSP',
    'điểm': 'SSSP'
}

def final_relation_check(text, df):
    re_result = re_model.predict(text)
    ner_result = ner.extract_document_metadata(text)

    # Safety checks
    if re_result is None or 'Span' not in re_result.columns or re_result['Span'].isna().all():
        return df

    # Get a clean span string
    span = str(re_result['Span'].iloc[0]).lower()
    span_tokens = re.findall(r'\w+', span)

    # Rule check
    if any(token in check_mask for token in span_tokens):
        meta = ner_result[['issue_date', 'title', 'document_id']].iloc[:1].reset_index(drop=True)
        rel = re_result.iloc[:1].reset_index(drop=True)
        combined = pd.concat([rel, meta], axis=1)
        df = pd.concat([df, combined], ignore_index=True)

    return df

In [6]:
def extract_sentences(text):
    sentences = []
    buffer = []

    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue

        buffer.append(line)

        if line.endswith(';'):
            sent = ' '.join(buffer).strip()

            # Remove header junk before first "Căn cứ"
            idx = sent.find("Căn cứ")
            if idx != -1:
                sent = sent[idx:].strip()

            sentences.append(sent)
            buffer = []

    return sentences

In [10]:
document = get_text_from_s3('thong_tu_bai_bo_ve_thue_2024.pdf')

def relation_extraction(text):
    
    df = pd.DataFrame(columns = ['Text', 'Self Root', 'Relation', 'Span', 'issue_date', 'title', 'document_id'])
    
    first = sent_tokenize(text)[0]

    sents = extract_sentences(first)
    
    for sent in sents:
        try:
            df = final_relation_check(sent, df)
        except Exception:
            print(f'No Relation in {sent}')
            
    return df

In [11]:
df_meta = ner.extract_document_metadata(document)

df_relation = relation_extraction(document)

No Relation in Theo đề nghị của Cục trưởng Cục Quản lý, giám sát chính sách thuế, phí và lệ phí;


In [13]:
print(sent_tokenize(document)[0])

BỘ TÀI CHÍNH  
-------  CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM  
Độc lập - Tự do - Hạnh phúc  
---------------  
Số: 52/2024/TT -BTC  Hà Nội, ngày 23 tháng 7 năm 2024  
  
THÔNG TƯ  
BÃI BỎ MỘT SỐ THÔNG TƯ CỦA BỘ TRƯỞNG BỘ TÀI CHÍNH TRONG LĨNH VỰC 
THUẾ  
Căn cứ Luật Ban hành văn bản quy phạm pháp luật ngày 22 tháng 6 năm 2015; Luật sửa đổi, bổ 
sung một số điều của Luật Ban hành văn bản quy phạm pháp luật ngày 18 tháng 6 năm 2020;  
Căn cứ Luật thuế giá trị gia tăng ngày 03 tháng 6 năm 2008, Luật sửa đổi, bổ sung một số điều 
của Luật thuế giá trị gia tăng ngày 19 tháng 6 năm 2013 và các văn bản hướng dẫn thi hành;  
Căn cứ Luật thuế thu nhập doanh nghiệp ngày 03 tháng 6 năm 2008, Luật sửa đổi, bổ sung một 
số điều của Luật thuế thu nhập doanh nghiệp ngày 19 tháng 6 năm 2013 và các văn bản hướng 
dẫn thi hành, Luật sửa đổi, bổ sung một số điều của các Luật về thuế ngày 26 tháng 11 năm 
2014 và các văn bản hướng dẫn thi hành;  
Căn cứ Luật thuế xuất khẩu, thuế nhập khẩu ngày 06 tháng 4 năm

In [14]:
df_meta

,issuer_department,issue_date,title,location,document_id,issuer,document_type
0,Bộ Tài Chính,23/07/2024,Thông Tư Bãi Bỏ Một Số Thông Tư Của Bộ Trưởng ...,Hà Nội,52/2024/TT-BTC,Cao Anh Tuấn,Thông Tư


In [15]:
df_relation

,Text,Self Root,Relation,Span,issue_date,title,document_id
0,Căn cứ Luật Ban hành văn bản quy phạm pháp luậ...,1,Pursuant,Luật Ban hành văn bản quy phạm pháp luật ngày ...,22/06/2015,Luật Ngày 22 Tháng 6,
1,Căn cứ Luật thuế giá trị gia tăng ngày 03 thán...,1,Pursuant,Luật thuế giá trị gia tăng ngày 03 tháng 6 năm,03/06/2008,"Luật Sửa Đổi , Bổ Sung Một Số Điều",
2,Căn cứ Luật thuế thu nhập doanh nghiệp ngày 03...,1,Pursuant,Luật thuế thu nhập doanh nghiệp ngày 03 tháng ...,03/06/2008,"Luật Sửa Đổi , Bổ Sung Một Số Điều",
3,"Căn cứ Luật thuế xuất khẩu, thuế nhập khẩu ngà...",1,Pursuant,"Luật thuế xuất khẩu, thuế nhập khẩu ngày 06 th...",06/04/2016,"Luật Thuế Xuất Khẩu, Thuế Nhập Khẩu Ngày 06 Th...",
4,Căn cứ Nghị định số 34/2016/NĐ -CP ngày 14 thá...,1,Pursuant,Nghị định số 34/2016/NĐ -CP ngày 14 tháng 5,14/05/2016,Nghị Định Số 34/2016/Nđ -Cp Ngày 14 Tháng 5 Nă...,34/2016/NĐ-CP
5,Căn cứ Nghị định số 154/2020/NĐ -CP ngày 31 th...,1,Pursuant,Nghị định số 154/2020/NĐ -CP ngày 31,31/12/2020,Nghị Định Số 154/2020/Nđ -Cp Ngày 31 Tháng 12 ...,154/2020/NĐ-CP
6,Căn cứ Nghị định số 14/2023/NĐ -CP ngày 20 thá...,1,Pursuant,Nghị định số 14/2023/NĐ -CP ngày 20,20/04/2023,Nghị Định Số 14/2023/Nđ -Cp Ngày 20 Tháng 4 Nă...,14/2023/NĐ-CP
